In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from InstructorEmbedding import INSTRUCTOR

/home/lee-j/github/supreme-parakeet/.venv/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [4]:
# Load the hkunlp/instructor-large model 
model = INSTRUCTOR('hkunlp/instructor-large')


load INSTRUCTOR_Transformer


max_seq_length  512


In [5]:
# Load the csv
dataset_path = csv_file = '../asr/cv-valid-dev.csv'
df = pd.read_csv(dataset_path)

In [6]:
# Define the hot words and their embeddings
hot_words = ["be careful", "destroy", "stranger"]
hot_word_embeddings = {word: model.encode(word) for word in hot_words}

In [ ]:
# Define a function to calculate similarity
def calculate_similarity(transcription):
    sent_embedding = model.encode(transcription)
    similarities = [np.cosine_similarity(sent_embedding, hot_word_embedding, dim=1).item() for hot_word_embedding in hot_word_embeddings.values()]
    return max(similarities)

In [ ]:
# Calculate similarity for each phrase in the dataset
similarities = []
for phrase in tqdm(df["sentence"]):
    similarity = calculate_similarity(phrase)
    similarities.append(similarity)

In [ ]:
# Add the similarity column to the dataset
df["similarity"] = [similarity > 0.7 for similarity in similarities]  # the threshold can be adjusted

In [ ]:
# Save the updated dataset
output_path = "cv-valid-dev-with-similarity.csv"
df.to_csv(output_path, index=False)
